In [1]:
import cv2 as cv 
import numpy as np
import scipy
import PIL.Image
import math
# caffe_root = "/home/yang/Workspace/TASK/keypoint/caffe_rtpose/"
# import sys
# sys.path.insert(0, caffe_root + 'python')
import caffe
import time
from config_reader import config_reader
import util

In [2]:
test_image = '../sample_image/ski.jpg'
#test_image = '../sample_image/upper.jpg'
#test_image = '../sample_image/upper2.jpg'
oriImg = cv.imread(test_image) # B,G,R order

In [3]:
param, model = config_reader()
multiplier = [x * model['boxsize'] / oriImg.shape[0] for x in param['scale_search']]

In [4]:
if param['use_gpu']: 
    caffe.set_mode_gpu()
    caffe.set_device(param['GPUdeviceNumber']) # set to your device!
else:
    caffe.set_mode_cpu()
net = caffe.Net(model['deployFile'], model['caffemodel'], caffe.TEST)

In [5]:
heatmap_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 19))
paf_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 38))

for m in range(len(multiplier)):
    scale = multiplier[m]
    imageToTest = cv.resize(oriImg, (0,0), fx=scale, fy=scale, interpolation=cv.INTER_CUBIC)
    imageToTest_padded, pad = util.padRightDownCorner(imageToTest, model['stride'], model['padValue'])
    print imageToTest_padded.shape

    net.blobs['data'].reshape(*(1, 3, imageToTest_padded.shape[0], imageToTest_padded.shape[1]))
    #net.forward() # dry run
    net.blobs['data'].data[...] = np.transpose(np.float32(imageToTest_padded[:,:,:,np.newaxis]), (3,2,0,1))/256 - 0.5;
    start_time = time.time()
    output_blobs = net.forward()
    print('At scale %d, The CNN took %.2f ms.' % (m, 1000 * (time.time() - start_time)))

    # extract outputs, resize, and remove padding
    heatmap = np.transpose(np.squeeze(net.blobs[output_blobs.keys()[1]].data), (1,2,0)) # output 1 is heatmaps
    heatmap = cv.resize(heatmap, (0,0), fx=model['stride'], fy=model['stride'], interpolation=cv.INTER_CUBIC)
    heatmap = heatmap[:imageToTest_padded.shape[0]-pad[2], :imageToTest_padded.shape[1]-pad[3], :]
    heatmap = cv.resize(heatmap, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv.INTER_CUBIC)
    
    paf = np.transpose(np.squeeze(net.blobs[output_blobs.keys()[0]].data), (1,2,0)) # output 0 is PAFs
    paf = cv.resize(paf, (0,0), fx=model['stride'], fy=model['stride'], interpolation=cv.INTER_CUBIC)
    paf = paf[:imageToTest_padded.shape[0]-pad[2], :imageToTest_padded.shape[1]-pad[3], :]
    paf = cv.resize(paf, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv.INTER_CUBIC)
     
    heatmap_avg = heatmap_avg + heatmap / len(multiplier)
    paf_avg = paf_avg + paf / len(multiplier)


(184, 200, 3)
At scale 0, The CNN took 649.45 ms.
(368, 392, 3)
At scale 1, The CNN took 1543.13 ms.
(552, 584, 3)
At scale 2, The CNN took 3410.82 ms.
(736, 784, 3)
At scale 3, The CNN took 5926.14 ms.


In [6]:
import scipy
print heatmap_avg.shape

#plt.imshow(heatmap_avg[:,:,2])
from scipy.ndimage.filters import gaussian_filter
all_peaks = []
all_peaks_lst = []
peak_counter = 0

for part in range(19-1):
    x_list = []
    y_list = []
    map_ori = heatmap_avg[:,:,part]
    map = gaussian_filter(map_ori, sigma=3)
    
    map_left = np.zeros(map.shape)
    map_left[1:,:] = map[:-1,:]
    map_right = np.zeros(map.shape)
    map_right[:-1,:] = map[1:,:]
    map_up = np.zeros(map.shape)
    map_up[:,1:] = map[:,:-1]
    map_down = np.zeros(map.shape)
    map_down[:,:-1] = map[:,1:]
    
    peaks_binary = np.logical_and.reduce((map>=map_left, map>=map_right, map>=map_up, map>=map_down, map > param['thre1']))
    peaks = zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0]) # note reverse
    all_peaks_lst += peaks
    peaks_with_score = [x + (map_ori[x[1],x[0]],) for x in peaks]
    id = range(peak_counter, peak_counter + len(peaks))
    peaks_with_score_and_id = [peaks_with_score[i] + (id[i],) for i in range(len(id))]

    all_peaks.append(peaks_with_score_and_id)
    peak_counter += len(peaks)
print all_peaks
print all_peaks_lst

(674, 712, 19)
[[(616, 44, 0.93386441469192505, 0), (303, 78, 0.93961541354656219, 1), (481, 98, 0.91290576756000519, 2), (68, 268, 0.93640249967575073, 3), (173, 280, 0.95367415249347687, 4)], [(614, 83, 0.88991190493106842, 5), (306, 113, 0.92324985563755035, 6), (458, 144, 0.79451495409011841, 7), (66, 302, 0.87793654203414917, 8), (170, 304, 0.92667004466056824, 9)], [(584, 89, 0.81856814026832581, 10), (272, 117, 0.8959868997335434, 11), (416, 146, 0.64621058851480484, 12), (39, 303, 0.87037493288516998, 13), (139, 304, 0.89320802688598633, 14)], [(545, 131, 0.91910834610462189, 15), (348, 142, 0.43601641868008301, 16), (259, 167, 0.74719075858592987, 17), (65, 341, 0.65085992217063904, 18), (137, 360, 0.79141350090503693, 19)], [(568, 170, 0.7006695568561554, 20), (283, 179, 0.66114842146635056, 21), (282, 180, 0.66213471442461014, 22), (93, 355, 0.27196971327066422, 23), (171, 376, 0.81103268265724182, 24)], [(645, 76, 0.87654848396778107, 25), (339, 109, 0.85844880342483521, 26

In [7]:
# find connection in the specified sequence, center 29 is in the position 15
limbSeq = [[2,3], [2,6], [3,4], [4,5], [6,7], [7,8], [2,9], [9,10], \
           [10,11], [2,12], [12,13], [13,14], [2,1], [1,15], [15,17], \
           [1,16], [16,18], [3,17], [6,18]]
# the middle joints heatmap correpondence
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], [19,20], [21,22], \
          [23,24], [25,26], [27,28], [29,30], [47,48], [49,50], [53,54], [51,52], \
          [55,56], [37,38], [45,46]]

In [8]:
connection_all = []
special_k = []
mid_num = 10

for k in range(len(mapIdx)):
    score_mid = paf_avg[:,:,[x-19 for x in mapIdx[k]]]
    candA = all_peaks[limbSeq[k][0]-1]
    candB = all_peaks[limbSeq[k][1]-1]
    nA = len(candA)
    nB = len(candB)
    indexA, indexB = limbSeq[k]
    if(nA != 0 and nB != 0):
        connection_candidate = []
        for i in range(nA):
            for j in range(nB):
                vec = np.subtract(candB[j][:2], candA[i][:2])
                norm = math.sqrt(vec[0]*vec[0] + vec[1]*vec[1])
                vec = np.divide(vec, norm)
                
                startend = zip(np.linspace(candA[i][0], candB[j][0], num=mid_num), \
                               np.linspace(candA[i][1], candB[j][1], num=mid_num))
                
                vec_x = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 0] \
                                  for I in range(len(startend))])
                vec_y = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 1] \
                                  for I in range(len(startend))])

                score_midpts = np.multiply(vec_x, vec[0]) + np.multiply(vec_y, vec[1])
                score_with_dist_prior = sum(score_midpts)/len(score_midpts) + min(0.5*oriImg.shape[0]/norm-1, 0)
                criterion1 = len(np.nonzero(score_midpts > param['thre2'])[0]) > 0.8 * len(score_midpts)
                criterion2 = score_with_dist_prior > 0
                if criterion1 and criterion2:
                    connection_candidate.append([i, j, score_with_dist_prior, score_with_dist_prior+candA[i][2]+candB[j][2]])

        connection_candidate = sorted(connection_candidate, key=lambda x: x[2], reverse=True)
        connection = np.zeros((0,5))
        for c in range(len(connection_candidate)):
            i,j,s = connection_candidate[c][0:3]
            if(i not in connection[:,3] and j not in connection[:,4]):
                connection = np.vstack([connection, [candA[i][3], candB[j][3], s, i, j]])
                if(len(connection) >= min(nA, nB)):
                    break

        connection_all.append(connection)
    else:
        special_k.append(k)
        connection_all.append([])

In [9]:
# last number in each row is the total parts number of that person
# the second last number in each row is the score of the overall configuration
subset = -1 * np.ones((0, 20))
candidate = np.array([item for sublist in all_peaks for item in sublist])

for k in range(len(mapIdx)):
    if k not in special_k:
        partAs = connection_all[k][:,0]
        partBs = connection_all[k][:,1]
        indexA, indexB = np.array(limbSeq[k]) - 1

        for i in range(len(connection_all[k])): #= 1:size(temp,1)
            found = 0
            subset_idx = [-1, -1]
            for j in range(len(subset)): #1:size(subset,1):
                if subset[j][indexA] == partAs[i] or subset[j][indexB] == partBs[i]:
                    subset_idx[found] = j
                    found += 1
            
            if found == 1:
                j = subset_idx[0]
                if(subset[j][indexB] != partBs[i]):
                    subset[j][indexB] = partBs[i]
                    subset[j][-1] += 1
                    subset[j][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]
            elif found == 2: # if found 2 and disjoint, merge them
                j1, j2 = subset_idx
                print "found = 2"
                membership = ((subset[j1]>=0).astype(int) + (subset[j2]>=0).astype(int))[:-2]
                if len(np.nonzero(membership == 2)[0]) == 0: #merge
                    subset[j1][:-2] += (subset[j2][:-2] + 1)
                    subset[j1][-2:] += subset[j2][-2:]
                    subset[j1][-2] += connection_all[k][i][2]
                    subset = np.delete(subset, j2, 0)
                else: # as like found == 1
                    subset[j1][indexB] = partBs[i]
                    subset[j1][-1] += 1
                    subset[j1][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]

            # if find no partA in the subset, create a new subset
            elif not found and k < 17:
                row = -1 * np.ones(20)
                row[indexA] = partAs[i]
                row[indexB] = partBs[i]
                row[-1] = 2
                row[-2] = sum(candidate[connection_all[k][i,:2].astype(int), 2]) + connection_all[k][i][2]
                subset = np.vstack([subset, row])

In [10]:
# delete some rows of subset which has few parts occur
deleteIdx = [];
for i in range(len(subset)):
    if subset[i][-1] < 4 or subset[i][-2]/subset[i][-1] < 0.4:
        deleteIdx.append(i)
subset = np.delete(subset, deleteIdx, axis=0)
print subset

[[  4.           9.          14.          19.          24.          29.
   34.          39.          43.          48.          53.          58.
   63.          68.          74.          79.          84.          88.
   29.24890465  18.        ]
 [  3.           8.          13.          18.          23.          28.
   33.          38.          44.          47.          52.          59.
   62.          67.          73.          78.          83.          87.
   27.0958793   18.        ]
 [  0.           5.          10.          15.          20.          25.
   30.          35.          40.          45.          50.          55.
   61.          66.          70.          75.          80.          85.
   31.09361054  18.        ]
 [  1.           6.          11.          17.          22.          26.
   31.          36.          41.          46.          51.          56.
   60.          65.          71.          76.          81.          86.
   28.09304559  18.        ]
 [  2.           7. 

In [11]:
human_grid = []
for human_i in subset:
    human_grid_i = []
    for i in range(18):
        body_part_index = int(human_i[i])
        if body_part_index == -1:
            grid = (0, 0)
        else:
            grid = all_peaks_lst[body_part_index]
        human_grid_i += [grid[0], grid[1]]
    human_grid.append(human_grid_i)
all_grid = np.array(human_grid)
print all_grid.shape
print human_grid

(5, 36)
[[173, 280, 170, 304, 139, 304, 137, 360, 171, 376, 200, 305, 203, 357, 189, 375, 155, 379, 137, 373, 196, 402, 192, 376, 241, 363, 170, 406, 167, 273, 180, 274, 156, 273, 190, 277], [68, 268, 66, 302, 39, 303, 65, 341, 93, 355, 94, 301, 122, 331, 99, 349, 39, 389, 80, 337, 111, 402, 80, 387, 113, 338, 132, 405, 61, 262, 75, 262, 49, 269, 83, 269], [616, 44, 614, 83, 584, 89, 545, 131, 568, 170, 645, 76, 659, 125, 661, 166, 615, 200, 620, 280, 616, 353, 651, 194, 649, 277, 639, 353, 608, 38, 622, 37, 596, 44, 631, 39], [303, 78, 306, 113, 272, 117, 259, 167, 282, 180, 339, 109, 353, 147, 301, 167, 296, 213, 310, 282, 324, 369, 339, 208, 344, 277, 295, 347, 296, 72, 309, 71, 285, 78, 318, 75], [481, 98, 458, 144, 416, 146, 348, 142, 283, 179, 502, 138, 541, 193, 568, 195, 436, 324, 509, 435, 531, 561, 465, 318, 463, 461, 411, 557, 466, 87, 488, 84, 437, 91, 0, 0]]
